<table>
<tr height="150px">
<th>
<img height="80px" margin="20px" src='https://www.archimedesai.gr/images/logo_en.svg' />
</th>
<th>
<img height="150px" src='https://stergioc.github.io/assets/img/logos.png' />
</th>
</tr>
</table>

<h1>Introduction to Machine Learning (Hands-on Tutorial)</h1>
<h3>Stergios CHRISTODOULIDIS & Maria VAKALOPOULOU</h3>


[![ML-tutorial](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stergioc/BioMed-AI-Summer-School/blob/master/ML/ml-tutorial.ipynb)

# Introduction

In this tutorial we will investigate a simple pipeline for extracting radiomic
features from MRI images and then we will use them for a classification task to discriminate between low-grade (LGG) and high-grade (HGG) glioma patients. The data are retrieved from the [BraTS dataset](https://www.med.upenn.edu/cbica/brats2019/data.html) and have been already lightly preprocessed for this tutorial.

### Documentation Links
- [SimpleITK](https://simpleitk.readthedocs.io/en/master/) - Used for loading and processing radiological data.
- [pyradiomics](https://pyradiomics.readthedocs.io/en/latest/) - Used for extracting features from radiological data.
- [Pandas](https://pandas.pydata.org/docs/) - Used for data organization
- [scikit-learn](https://scikit-learn.org/stable/user_guide.html) - Used for training machine learning models


In the following code block the necessary packages will be installed and imported. Moreover, the different files that are required for the purposes of this tutorial will be downloaded.

In [ ]:
# Installing necessary libraries
!pip install SimpleITK pyradiomics

# Downloading the required files
!wget https://nextcloud.centralesupelec.fr/s/jCeq7GrMGyyYKSn/download -O data.zip
!unzip -o data.zip

# Importing the necessary libaries
import glob
import pandas as pd
import seaborn as sns
import SimpleITK as sitk
import matplotlib.pyplot as plt
# This module is used for interaction with pyradiomics
from radiomics import featureextractor  

# Part I - Loading and Visualization of data

In this first part we will load and visualize the data we downloaded. The data are MRI images of the brain with tumoral regions. For each patient a number of different modalities are available (T1, T2, T1ce, FLAIR) together with a segmentation map.

In [ ]:
# Retrieves the directories of the downloaded MRI images
t1_file_list = sorted(glob.glob('data/mri/**/**/*_t1.nii.gz'))
t2_file_list = sorted(glob.glob('data/mri/**/**/*_t2.nii.gz'))
t1ce_file_list = sorted(glob.glob('data/mri/**/**/*_t1ce.nii.gz'))
flair_file_list = sorted(glob.glob('data/mri/**/**/*_flair.nii.gz'))
seg_file_list = sorted(glob.glob('data/mri/**/**/*_seg.nii.gz'))

# Organize all the data in a DataFrame
database = pd.DataFrame({
    'T1': t1_file_list,
    'T2': t2_file_list,
    'T1ce': t1ce_file_list,
    'FLAIR': flair_file_list,
    'Seg': seg_file_list
})
database.head()

In [ ]:
IDX = 10

# Load a single MRI file using SimpleITK
img_sitk = sitk.ReadImage(database.at[IDX,'T1'])
img_np = sitk.GetArrayFromImage(img_sitk)

# Load a single MRI file using SimpleITK
seg_sitk = sitk.ReadImage(database.at[IDX,'Seg'])
seg_np = sitk.GetArrayFromImage(seg_sitk)

# Shows the dimensions of the MRI image
print(f"Shape of the image: {img_np.shape}")
print(f"Shape of the segmentation: {seg_np.shape}")

There are $155$ slices in the volume, where each slice is $240\times240$ pixel image. Similary the segmentation image have the same dimensions. Let's visualise some of them.

In [ ]:
# Visualize a single slide of the MRI with the segmentation mask overlaid
SLIDE = 80

plt.figure()
plt.imshow(img_np[SLIDE,...], cmap='gray')
plt.imshow(seg_np[SLIDE,...], alpha=0.5)
plt.axis('off')
plt.show()

In [ ]:
# Visualize multiple slices of the MRI image with the segmentation mask overlaid
fig, axes = plt.subplots(ncols=5, nrows=4, figsize=(7, 5))

for i in range(0,img_np.shape[0],8):
  axes.ravel()[i//8].set_title(f'Slice: {i}')
  axes.ravel()[i//8].imshow(img_np[i,...], cmap='gray')
  axes.ravel()[i//8].imshow(seg_np[i,...], alpha=0.5)
  axes.ravel()[i//8].axis('off')

plt.tight_layout()
plt.show()

As already mentioned for this specific database there are multiple modalities available, let's visualize all the modalities for a single patient.

In [ ]:
# Load all MRI protocols and the segmentation
t1_np = sitk.GetArrayFromImage(sitk.ReadImage(database.at[IDX,'T1']))
t2_np = sitk.GetArrayFromImage(sitk.ReadImage(database.at[IDX,'T2']))
t1ce_np = sitk.GetArrayFromImage(sitk.ReadImage(database.at[IDX,'T1ce']))
flair_np = sitk.GetArrayFromImage(sitk.ReadImage(database.at[IDX,'FLAIR']))
seg_np = sitk.GetArrayFromImage(sitk.ReadImage(database.at[IDX,'Seg']))

all_images = {
    'T1': t1_np, 
    'T2': t2_np, 
    'T1ce': t1ce_np, 
    'FLAIR': flair_np, 
    'Segmentation': seg_np
}

# Visualize all the protocols
fig, axes = plt.subplots(ncols=5, figsize=(15, 5))
for i, (k,img) in enumerate(all_images.items()):
  colormap = None if i==4 else 'gray'
  axes[i].imshow(img[SLIDE,...], cmap=colormap)
  axes[i].set_title(k)
  axes[i].axis('off')
plt.tight_layout()

# Part II - Data Preprocessing

To stabilise the downstream feature extraction and analysis, we first normalise the data. For input image $I$, we create normalised image $I_z$ by normalising each pixel $x_{ij}$ according to,

$$I_z^{ij} = \frac{I^{ij} - \mu_I}{\sigma_I}.$$

Because of the abundance of background values in each volume, we compute $\mu_I$ and $\sigma_I$ only over the pixels of $I$ above a certain threshold.

In [ ]:
# Histogram of the pixel values for each modality
plt.hist([
    t1_np.ravel(),
    t2_np.ravel(),
    t1ce_np.ravel(),
    flair_np.ravel()
    ], label=['T1', 'T2', 'T1ce','FLAIR'],
    histtype='stepfilled', alpha=0.3, density=True, bins=40, ec="k")

plt.title('Pixel Values Histogram for all Modalities')
plt.xlabel('Intensity Value')
plt.ylabel('Pixel Count')
plt.legend()
plt.yscale('log')
plt.show()

### Excerise 1 - MRI Image Loading and Standardization: 

Write a function `load_img_norm` that will load and normalize  an image in order to be used in the following parts of the tutorial. The function will take as an input arguments the image `image_dir` and the `threshold`. The function will need to return the normalized sitk object and a numpy array by using only the elements of the volume with values larger than the threshold.


You can use the necessary function from the previous code blocks as well as the following functions: 
 - `<SimpleITK object> = sitk.GetImageFromArray(<numpy array>)`
 - `<SimpleITK object>.CopyInformation(<SimpleITK object>)`

In [ ]:
def load_img_norm(image_dir, threshold=0):
  ### START CODE HERE ###

  ### END CODE HERE ###
  return normalized_sitk, normalized_np

In [ ]:
# Histogram of the pixel values for each modality
IDX = 10
plt.hist([
    load_img_norm(database.at[IDX,'T1'])[1].ravel(),
    load_img_norm(database.at[IDX,'T2'])[1].ravel(),
    load_img_norm(database.at[IDX,'T1ce'])[1].ravel(),
    load_img_norm(database.at[IDX,'FLAIR'])[1].ravel(),
    ], label=['T1_norm', 'T2_norm', 'T1ce_norm','FLAIR_norm'],
    histtype='stepfilled', alpha=0.3, density=True, bins=40, ec="k")

plt.title('Pixel Values Histogram for all Modalities')
plt.xlabel('Intensity Value')
plt.ylabel('Pixel Count')
plt.legend()
plt.yscale('log')
plt.show()

# Part III - Radiomics Feature Extraction

### Excerise 2 - Radiomics Feature Extraction: 

Now that we have prepared our image loaders, we can proceed to extract the radiomics features. We use the default settings of the extractor. Let us extract features for a single patient, under the T1 modality, for the full tumour region.

You can use the excecute function from the `RadiomicsFeatureExtractor()` class that it is available in the `featureextractor` that we have already loaded.

Check the documentation [here](https://pyradiomics.readthedocs.io/en/latest/radiomics.html#radiomics.featureextractor.RadiomicsFeatureExtractor).

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

print(pd.Series(result))

# Part IV - Feature Exploration and Visualization

The first step in applying machine learning is to understand the data, so as to be able to identify a good target to learn, as well as an appropriate learning algorithm among many alternatives.

The analysis above has been applied on few patients only. Since our time in this practical session is limited, the features on the entire BraTS dataset has been extracted into a csv file located at `./data/radiomics.csv`. This file contains one row per patient per input modality (T1, T1ce, T2, flair). With the feaures from the full cohort, we can begin to explore the data.

In [ ]:
data = pd.read_csv('data/radiomics.csv')
display(data)

### Pairwise joint distributions

We can see the join distribution of any pair of columns/attributes/variables/features by using the pairplot function. We do this for the first 5 features. Each datapoint is a patient and the colour indicates the disease class.

In [ ]:
# filter one sequence type and a segmentation type (you can change the sequence type and segmentation type)
d = data[(data['sequence']=='t1ce')].iloc[:, 2:8] 
sns.pairplot(d, hue='label', size=2.5)
plt.tight_layout()
plt.show()

### Feature Correlation Heatmap

In [ ]:
# filter one sequence type and a segmentation type (you can change the sequence type and segmentation type)
d = data[(data['sequence']=='t1')].iloc[:, 3:]
sns.clustermap(d.corr())
plt.show()

# Part V - Data Preparation

In [ ]:
# Reshapes the dataframe such as each row to have all the features for a patient
data = data.pivot_table(index=['patient', 'label'],
                                columns=['sequence'],
                                values=data.columns[3:])
data.columns = ['_'.join(col).strip() for col in data.columns.values]
data.reset_index(level=1, inplace=True)

# Convert LGG into class 0 and HGG into class 1
data.loc[data['label'] == 'HGG', 'label'] = 1
data.loc[data['label'] == 'LGG', 'label'] = 0

display(data)

### Train-Test Splitting

Let's now use <code>train_test_split</code> from the function in scikit-learn to divide features data (x_data) and target data (y_data) even further into train and test. Here we will have 30% of the data for the test set. It is also a good practice to define a random state for reproducible results.

*Note: Stratify parameter in the function makes the equal proportion in the split. For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y_data will make sure that your random split has 25% of 0's and 75% of 1'*

In [ ]:
from sklearn.model_selection import train_test_split

x_data, y_data = data.drop(columns='label'), data['label'].astype(int).to_numpy()
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data ,test_size = 0.3, random_state=123, stratify=y_data)

print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

### Exercise 3 - Radiomics Features Preprocessing

As we have already identified the differnt radiomics features are following different distributions ($\mu$,$\sigma$). It is important for most classifiers all the features to fall on similar value ranges. For this purpose we will need to normalize our features. 

You can use the available preprocessing tools that are available within the scikit-learn library for this purpose, e.g., [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) or [MinMax](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).



*Tips: You can apply multiple functions as a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) using the pipeline class. See [here](https://scikit-learn.org/stable/modules/preprocessing.html) for more preprocessing guidelines.*





In [ ]:
### START CODE HERE ###

### END CODE HERE ###

### Exersise 4 - Train a classifier

With all the features extracted and preprocessed, we can proceed to training our first machine learning model. You can select any model from the scikit-learn library and train it.

You can find all the available models [here](https://scikit-learn.org/stable/supervised_learning.html).

*Tips: Start simple using the default parameters of a [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) classifier or a [random forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).*

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

### Performance Measure

Once the model is trained, i.e. `fit` function is done, it can be used to classify any input sample with the correct shape. Notably, training and validation accuracies can be obtained with other performance indicators:

In [ ]:
accuracy_train = clf.score(X=x_train_norm, y=y_train)
probs = clf.predict_proba(x_train)
accuracy_test = clf.score(X=x_test_norm, y=y_test)
print(f'Training accuracy: {accuracy_train}')
print(f'Test accuracy: {accuracy_test}')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

predictions = clf.predict(x_test_norm)

cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['LGG','HGG'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.grid(False)
plt.show()